In [1]:
import sys
sys.path.append('/home/franzihe/Documents/Python/Thesis/')#MEPS/')

import pandas as pd
import numpy as np
import netCDF4
import createFolder as cF


In [2]:
station = 'Haukeliseter'

In [3]:
txt_dir = '../../../Data/Retrieval/%s/' %station

In [4]:
m = ['10', '11', '12', '01', '02', '03']

In [5]:
txt_filename = 'minute_surface_acc_Haukeli.dat.txt'



In [6]:
def get_netCDF_variable(f, var_name, var, dim):
    v_0m = f.createVariable(varname=var_name, datatype=var.dtype, dimensions=dim, zlib=True)
    v_0m[:] = var[:]
    return(v_0m)

In [7]:
for month in m:
    if month == '10':
        t = np.arange(16,32)
    if month == '11':
        t = np.arange(1,31)
    if month == '12' or month == '01' or month == '03':
        t = np.arange(1,32)
    if month == '02':
        t = np.arange(1,29)
    if month == '10' or month == '11' or month == '12':
        year = '2016'
    if month == '01' or month == '02' or month == '03':
        year = '2017'
        
    for day in t:
        
        if day < 10:
            day = '0%s' %day
        else:
            day = '%s' %day
        
        Date = int(year+month+day)
        
        
        Haukeli_p = pd.read_fwf(txt_dir + 'pulsed_'+txt_filename, header = 1)
        idx_p = np.where(Haukeli_p['Unnamed: 0'] == Date)
        
        if np.array(idx_p).shape[1] == 1:
                precip_24 = (Haukeli_p['Unnamed: 1'][idx_p[0][0]])
                precip_24 = np.array([precip_24])
                
                #read in the surface snowfall rate for each minute
                column0 = Haukeli_p['Unnamed: 0'][idx_p[0][0]+1:int(idx_p[0][0]+1440/6+1)]
                column1 = Haukeli_p['Unnamed: 1'][idx_p[0][0]+1:int(idx_p[0][0]+1440/6+1)]
                column2 = Haukeli_p['Unnamed: 2'][idx_p[0][0]+1:int(idx_p[0][0]+1440/6+1)]
                column3 = Haukeli_p['Unnamed: 3'][idx_p[0][0]+1:int(idx_p[0][0]+1440/6+1)]
                column4 = Haukeli_p['Unnamed: 4'][idx_p[0][0]+1:int(idx_p[0][0]+1440/6+1)]
                column5 = Haukeli_p['Unnamed: 5'][idx_p[0][0]+1:int(idx_p[0][0]+1440/6+1)]
        
                ret_pulsed = pd.concat([column0, column1, column2, column3, column4, column5], axis =1)
                ret_pulsed = np.array(ret_pulsed)
                ret_pulsed = ret_pulsed.flatten()/ret_pulsed.flatten().shape[0]
                
        Haukeli_u = pd.read_fwf(txt_dir + 'upslope_'+txt_filename, header = 1)
        idx_u = np.where(Haukeli_u['Unnamed: 0'] == Date)
        
        if np.array(idx_u).shape[1] == 1:
                precip_24 = (Haukeli_u['Unnamed: 1'][idx_u[0][0]])
                precip_24 = np.array([precip_24])
                
                #read in the surface snowfall rate for each minute
                column0_u = Haukeli_u['Unnamed: 0'][idx_u[0][0]+1:int(idx_u[0][0]+1440/6+1)]
                column1_u = Haukeli_u['Unnamed: 1'][idx_u[0][0]+1:int(idx_u[0][0]+1440/6+1)]
                column2_u = Haukeli_u['Unnamed: 2'][idx_u[0][0]+1:int(idx_u[0][0]+1440/6+1)]
                column3_u = Haukeli_u['Unnamed: 3'][idx_u[0][0]+1:int(idx_u[0][0]+1440/6+1)]
                column4_u = Haukeli_u['Unnamed: 4'][idx_u[0][0]+1:int(idx_u[0][0]+1440/6+1)]
                column5_u = Haukeli_u['Unnamed: 5'][idx_u[0][0]+1:int(idx_u[0][0]+1440/6+1)]
        
                ret_upslope = pd.concat([column0_u, column1_u, column2_u, column3_u, column4_u, column5_u], axis =1)
                ret_upslope = np.array(ret_upslope)
                ret_upslope = ret_upslope.flatten()/ret_upslope.flatten().shape[0]
         
        ### write netCDF file
        cF.createFolder(txt_dir+txt_filename.split()[0][0:-16]+'/')
        fw = netCDF4.Dataset(txt_dir+txt_filename.split()[0][0:-16]+'/%s.nc' %Date, 'w')
        ### create dimensions
        fw.createDimension('time', ret_pulsed.shape[0])
        
        ret_pulsed_w = get_netCDF_variable(fw, 'pulsed_sfc_acc', ret_pulsed, ('time',))
        ret_upslope_w = get_netCDF_variable(fw, 'upslope_sfc_acc', ret_upslope, ('time',))
        
        fw.close()